In [47]:
import os
import csv
import shutil
import matplotlib.pyplot
import pickle
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras import optimizers
from keras.applications import VGG16
from PIL import Image



In [44]:


#create strings for directory access
baseDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset'


trainDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\TRAIN'
smallTrainDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\SMALLTRAIN'
finalTrainDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\FINALTRAIN'


testDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\TEST'
smallTestDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\SMALLTEST'


subdirectoriesList = [ 'EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL' ]



In [31]:


#make small train directories because the learning time is too long

# os.chdir( baseDirectory )
# os.mkdir( 'SMALLTRAIN' )

# os.chdir( smallTrainDirectory )
# os.mkdir( 'EOSINOPHIL' )
# os.mkdir( 'LYMPHOCYTE' )
# os.mkdir( 'MONOCYTE' )
# os.mkdir( 'NEUTROPHIL' )


# os.chdir( trainDirectory )
# for subdirectory in subdirectoriesList:
#     os.chdir( trainDirectory + '\\' + subdirectory )
#     fileNames = os.listdir()
    
#     for fileNameIndex in range( 600 ):
#         fileName = fileNames[ fileNameIndex ]
#         shutil.copy( fileName, smallTrainDirectory + '\\' + subdirectory )
        
    
    

#make small test directory because the learning time is too long
# os.chdir( baseDirectory )
# os.mkdir( 'SMALLTEST' )

# os.chdir( smallTestDirectory )
# os.mkdir( 'EOSINOPHIL' )
# os.mkdir( 'LYMPHOCYTE' )
# os.mkdir( 'MONOCYTE' )
# os.mkdir( 'NEUTROPHIL' )


# os.chdir( testDirectory )
# for subdirectory in subdirectoriesList:
#     os.chdir( testDirectory + '\\' + subdirectory )
#     fileNames = os.listdir()
    
#     for fileNameIndex in range( 250 ):
#         fileName = fileNames[ fileNameIndex ]
#         shutil.copy( fileName, smallTestDirectory + '\\' + subdirectory )



In [45]:

#create generators for images
finalTrainGenerator = ImageDataGenerator(rescale = (1/255))

testGenerator = ImageDataGenerator(rescale = (1/255))

trainGenerator = ImageDataGenerator(rescale = (1/255))



#create directory iterators from generators
trainDirectoryIterator = trainGenerator.flow_from_directory(directory = smallTrainDirectory,
                                                            target_size = (128, 128),
                                                            class_mode = 'categorical',
                                                            batch_size = 128)


finalTrainDirectoryIterator = finalTrainGenerator.flow_from_directory(directory = finalTrainDirectory,
                                                                      target_size = (128, 128),
                                                                      class_mode = 'categorical',
                                                                      batch_size = 128)


testDirectoryIterator = testGenerator.flow_from_directory(directory = testDirectory,
                                                          target_size = (128, 128),
                                                          class_mode = 'categorical',
                                                          batch_size = 128)




Found 2400 images belonging to 4 classes.
Found 9957 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [51]:

#create the convolutional base 
convolutionalBase = VGG16( weights = 'imagenet',
                           include_top = False,
                           input_shape = ( 128, 128, 3 ) )


#freeze the convolutional base
convolutionalBase.trainable = False 


#create the sequential model with the VGG16 convolutional base
sequentialModel = Sequential()
sequentialModel.add( convolutionalBase )
sequentialModel.add( Flatten() )
sequentialModel.add( Dropout( 0.3 ) )
sequentialModel.add( Dense( 256, activation = 'relu' ) )
sequentialModel.add( Dense( 4, activation = 'sigmoid' ) )


#compile the model
sequentialModel.compile( optimizer = optimizers.RMSprop( lr = 1e-4 ),
                       loss = 'categorical_crossentropy',
                       metrics = [ 'acc' ] )


#train the model
sequentialModelHistory = sequentialModel.fit_generator( generator = finalTrainDirectoryIterator,
                                                         steps_per_epoch = 78,
                                                         epochs = 60,
                                                         verbose = True,
                                                         validation_data = testDirectoryIterator,
                                                         validation_steps = 20 )

#save the model
sequentialModel.save( 'BloodTypeClassifierUsingVGG16Base.h5' )

#retrieve the training history
sequentialModelHistory = sequentialModelHistory.history

#save the training history to file
os.chdir( baseDirectory )
with open('trainHistoryDictionary.obj', 'wb') as fileToPickle:
        pickle.dump(sequentialModelHistory, fileToPickle)






Epoch 1/3
3/3 [==============================] - 9s 3s/step - loss: 1.4675 - acc: 0.2474 - val_loss: 1.3862 - val_acc: 0.2525
Epoch 2/3
3/3 [==============================] - 8s 3s/step - loss: 1.3807 - acc: 0.2812 - val_loss: 1.3804 - val_acc: 0.2939
Epoch 3/3
3/3 [==============================] - 8s 3s/step - loss: 1.3850 - acc: 0.2656 - val_loss: 1.3737 - val_acc: 0.2996


In [56]:
# with open( 'trainHistoryDictionary.obj', 'rb' ) as inputHistory:
#           trainHistory = pickle.load(inputHistory)
        
   

{'val_loss': [1.386176065023903, 1.3803784477217014, 1.3736656622854064], 'acc': [0.24739583333333334, 0.28125, 0.265625], 'loss': [1.4674618244171143, 1.3806888659795125, 1.3850034077962239], 'val_acc': [0.2525130679114161, 0.2939284278366716, 0.2995577002199577]}
